### Import Libraries

In [8]:
pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 672.2 kB/s eta 0:00:02
   -------------------- ------------------- 0.8/1.5 MB 838.9 kB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 838.9 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 699.0 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 699.0 kB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 645.3 kB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 645.3 kB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 640.7 kB/s  0:00:02

   ---------------------------------------- 0/4 [tqdm]
   ---------------------------------------- 0/4


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd 
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import json

In [11]:
df = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [12]:
df.head()

,sentence,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [13]:
df.shape

(1000, 2)

In [18]:
nltk.download('stopwords')
# Mengubah seluruh text kedalam bentuk lowercase
df['sentence'] = df['sentence'].str.lower()
 
# Menghilangkan stopwords
stop_word = set(stopwords.words('english'))

 
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
 
# Melakukan split dataset
sentence = df['sentence'].values
label = df['label'].values
 
sentence_train, sentence_test, label_train, label_test = train_test_split(sentence, label, test_size=0.2, shuffle=False)
 
# Membuat tokenisasi
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter untuk menghilangkan symbols
 
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)
 
tokenizer.fit_on_texts(sentence_train)
 
# Menyimpan word_index kedalam sebuah file json
word_index = tokenizer.word_index
 
with open('word_index.json', 'w') as fp:
    json.dump(word_index, fp)
 
# Membuat sequences dan melakukan padding
train_sekuens = tokenizer.texts_to_sequences(sentence_train)
test_sekuens = tokenizer.texts_to_sequences(sentence_test)
 
train_padded = pad_sequences(train_sekuens,
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

# Membuat model
model = tf.keras.Sequential([
    Embedding(2000, 20, input_length=20),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
 
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
 
# Train model
num_epochs = 30
history = model.fit(train_padded, label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, label_test),
                    verbose=1)

<>:18: SyntaxWarning: invalid escape sequence '\]'
<>:18: SyntaxWarning: invalid escape sequence '\]'
C:\Users\Rafa\AppData\Local\Temp\ipykernel_15516\1939893465.py:18: SyntaxWarning: invalid escape sequence '\]'
  filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter untuk menghilangkan symbols
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rafa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.5587 - loss: 0.6898 - val_accuracy: 0.2400 - val_loss: 0.7345
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5650 - loss: 0.6836 - val_accuracy: 0.2400 - val_loss: 0.7672
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5650 - loss: 0.6796 - val_accuracy: 0.2400 - val_loss: 0.7715
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5650 - loss: 0.6759 - val_accuracy: 0.2400 - val_loss: 0.7888
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5913 - loss: 0.6619 - val_accuracy: 0.2600 - val_loss: 0.7682
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7275 - loss: 0.6103 - val_accuracy: 0.3600 - val_loss: 0.7383
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8163 - loss: 0.5073 - val_accuracy: 0.8350 - val_loss: 0.5292
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9050 - loss: 0.3674 - val_accuracy: 0.8300 - val_loss